# Consecutive Groups

The goal is to convert a sequence of things, say integers, into a sequence of consequence groups. Given a list like this:

In [111]:
r = [1, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 20]

We want to convert it into something like:

In [112]:
[[1, 1], [4, 5], [7, 10], [12, 16], [20, 20]]

[[1, 1], [4, 5], [7, 10], [12, 16], [20, 20]]

or perhaps

In [113]:
[[1], [4, 5], [7, 10], [12, 16], [20]]

[[1], [4, 5], [7, 10], [12, 16], [20]]

Pre-existing solutions

The easy way is to just install the [more-itertools](https://more-itertools.readthedocs.io/en/stable/api.html#more_itertools.consecutive_groups) package, which has a method that gets you very close to this:

In [114]:
from more_itertools import consecutive_groups
list(map(list, consecutive_groups(r)))

[[1], [4, 5], [7, 8, 9, 10], [12, 13, 14, 15, 16], [20]]

In a second pass we could remove the inner items in the groups that have more than two entries.  But what are our options for doing it in one pass?

Starting with the naive approach first:

In [115]:
def make_ranges_single(r: List[int]):
    ranges = []
    for a in r:
        if not ranges or not ranges[-1]:
            ranges.append([a])
            continue
            
        if a == ranges[-1][-1] + 1:
            ranges[-1].append(a)
        else:
            ranges.append([a])
    return ranges        

In [116]:
print(make_ranges_single(r))

[[1], [4, 5], [7, 8, 9, 10], [12, 13, 14, 15, 16], [20]]


Gets us back to the `consecutive_groups` solution from _more-itertools_.  But we want to exclude the middle bits of groups that have a range, so

In [117]:
def make_ranges_single2(r: List[int]):
    ranges = []
    for a in r:
        if not ranges or not ranges[-1]:
            ranges.append([a])
            continue
            
        if a == ranges[-1][-1] + 1:
            if len(ranges[-1]) == 1:
                ranges[-1].append(a)
            else:
                # Overwrite the second element of this group
                ranges[-1][-1] = a
        else:
            ranges.append([a])
    return ranges   

In [118]:
print(make_ranges_single2(r))

[[1], [4, 5], [7, 10], [12, 16], [20]]


## functools.reduce

It occurred to me that this process looks very much like a reduction. Can we use the `reduce` function to do the same job?

In [119]:
from functools import reduce

In [120]:
def f(ranges, a):
    """The body of this function is identical to the for-loop in 
    the previous example!"""
    if not ranges or not ranges[-1]:
        ranges.append([a])
        return ranges
    
    if a == ranges[-1][-1] + 1:
        if len(ranges[-1]) == 1:
            ranges[-1].append(a)
        else:
            ranges[-1][-1] = a
    else:
        ranges.append([a])
    return ranges

In [121]:
print(f'r={r}')
print(reduce(f, r, []))

r=[1, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 20]
[[1], [4, 5], [7, 10], [12, 16], [20]]


We get the same using `reduce`.  Since the body of the reducer and the for-loop approach is the same, do we expect any performance difference?

### Timings

In [122]:
import sys
print('Python version:', sys.version, end='\n\n')

%timeit make_ranges_single2(r)
%timeit reduce(f, r, [])

Python version: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

2.91 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.54 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


There doesn't seem to be much difference, but our sequence is very short.  Let's make a larger sequence and try again.

In [123]:
from random import sample

bigr = sorted(set(sample(range(1_000_000), 500000)))
print(bigr[:10], '...', bigr[-10:])

[1, 5, 6, 8, 11, 13, 14, 15, 16, 17] ... [999982, 999984, 999986, 999988, 999989, 999993, 999994, 999995, 999996, 999999]


In [124]:
%timeit make_ranges_single2(bigr)
%timeit reduce(f, bigr, [])

151 ms ± 7.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
174 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


And the content looks like this:

In [125]:
ranges = make_ranges_single2(bigr)

In [126]:
print(ranges[:100])

[[1], [5, 6], [8], [11], [13, 17], [19], [21, 25], [29], [32], [34, 35], [40, 41], [43, 46], [48], [51, 52], [58], [63], [67, 68], [71], [76], [79], [84], [86], [90, 93], [95, 99], [101, 102], [105], [109], [111], [115, 118], [120, 122], [124, 125], [130, 131], [135], [140, 141], [145, 149], [152, 153], [157, 158], [160], [162, 163], [166], [168, 169], [172, 173], [175, 176], [180, 183], [185], [187, 188], [192], [194, 197], [199, 201], [204, 209], [213, 214], [216, 217], [220], [223], [229], [231, 233], [235, 236], [239], [242], [244], [246, 247], [251, 252], [256, 257], [259], [263], [270, 271], [273], [275], [280], [282, 283], [287, 289], [293], [296], [299, 301], [304, 305], [307, 308], [311, 313], [316], [320], [323], [325], [328], [337], [339, 340], [345, 347], [351, 354], [356, 362], [365], [371, 372], [374, 376], [378], [380, 381], [383, 384], [387, 391], [394], [396], [402], [406, 408], [412, 413], [415, 420]]


The more sparse _or_ dense the array is, the more compact will be the final representation. For example, if the input data has only 10 elements out of a pool of a million, you typically get out 10 groups of single elements:

In [127]:
make_ranges_single2(sorted(set(sample(range(1_000_000), 10))))

[[123475],
 [330919],
 [401559],
 [455704],
 [567898],
 [605655],
 [668479],
 [812333],
 [860165],
 [899989]]

However, if your input sequence is almost completely full, say only 10 elements short of the full million element, you also get a sparse representation:

In [128]:
big_ranges = make_ranges_single2(sorted(set(sample(range(1_000_000), 1_000_000 - 10))))
print(big_ranges)

[[0, 12278], [12280, 127465], [127467, 219925], [219927, 295128], [295130, 313021], [313023, 537491], [537493, 557266], [557268, 657134], [657136, 767071], [767073, 967509], [967511, 999999]]


## Applications

I became interested in this because of an application where I wanted to generate `BETWEEN` clauses for SQL statements, given a potentially large sequence of integers which may have many consecutive runs. Using the previously calculated value as an example, we might us it something like this:

In [129]:
from string import Template

tmpl = Template('''select * 
from table
where
  $betweeners
''')

ranges = make_ranges_single2(r)

betweeners = '\n  or '.join(
    f'id is between {a} and {b[0]}' if b else f'id = {a}' 
        for a, *b in ranges
)
print(tmpl.safe_substitute(betweeners=betweeners))

select * 
from table
where
  id = 1
  or id is between 4 and 5
  or id is between 7 and 10
  or id is between 12 and 16
  or id = 20



With a different format string, you can output the information differently:

In [130]:
range_summary = ','.join(f'{a}-{b[0]}' if b else f'{a}' for a, *b in ranges)
print(range_summary)

1,4-5,7-10,12-16,20


And given that string representation, to get back to the full sequence:

In [131]:
def reconstruct(range_summary: str):
    for group in range_summary.split(','):
        a, _, b = group.partition('-')
        if not b:
            yield int(a)
        else:
            yield from range(int(a), int(b) + 1)
            
remade = list(reconstruct(range_summary))
print(remade)
print('Same as original?', r == remade)

[1, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16, 20]
Same as original? True
